In [13]:
%pip install pandas
%pip install matplotlib


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached pyparsing-3.2.1-py3-none-any.whl.metadata (5.0 kB)
   ---------------------------------------- 0.0/8.0 MB ? eta -:--:--
   ---------------- ----------------------- 3.4/8.0 MB 20.2 MB/s eta 0:00:01
   ----------------------------------- ---- 7.1/8.0 MB 16.8 MB/s eta 0:00:01
   ---------------------------------------- 8.0/8.0 MB 14.6 MB/s eta 0:00:00
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 24.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ---------------------------------------- 2.6/2.6 MB 25.2 MB/s eta 0:00:00
Using cached pyparsing-3.2.1-py3-none-any.whl (107 kB)
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#pd.to_datetime(pd.read_csv('/content/DAT_ASCII_EURUSD_M1_2000.csv', delimiter=';').iloc[:,0].str[-6:],unit = 's')

In [17]:
data = pd.read_csv('c:/Users/estifo/Downloads/gamma-capture-project/data/DAT_ASCII_EURUSD_M1_2006.csv', header=None, sep=';')

In [ ]:
#data = pd.read_csv('/content/DAT_ASCII_EURUSD_M1_2000.csv')

In [ ]:
data

In [ ]:
data.shape

In [ ]:
# Rename columns for clarity
data.columns = ['DateTime', 'Open', 'High', 'Low', 'Close', 'Volume']

In [ ]:
data.tail(4)

In [ ]:
data.iloc[:10]

In [ ]:
# Convert 'DateTime' to datetime format
data['DateTime'] = pd.to_datetime(data['DateTime'], format='%Y%m%d %H%M%S')

In [ ]:
# Identify duplicate rows based on the 'DateTime' column
duplicates = data[data.duplicated(subset=['DateTime'], keep=False)]

# Get the indices of duplicate rows
duplicate_indices = duplicates.index.tolist()
print("Indices of Duplicate Rows (Based on DateTime Column):")
print(duplicate_indices)

In [ ]:
# Could not see any duplicated row
data.iloc[308461:308480]

In [ ]:
data.tail()

In [ ]:
data.shape

In [ ]:
data.set_index('DateTime', inplace=True)

In [ ]:
data

In [ ]:
# Ensure 'DateTime' is the index
data.index = pd.to_datetime(data.index)

# Generate a full range of 1-minute timestamps
full_range = pd.date_range(start=data.index.min(), end=data.index.max(), freq='1T')


In [ ]:
# Find missing timestamps
missing_timestamps = full_range.difference(data.index)

# Display missing timestamps
print(f"Missing Timestamps:\n{missing_timestamps}")


In [ ]:
data.iloc[35:45]

In [ ]:
data.iloc[50:60]

In [ ]:
missing_timestamps.shape

In [ ]:
# Convert missing timestamps to a DataFrame
missing_df = pd.DataFrame(missing_timestamps, columns=['DateTime'])

In [ ]:
missing_df['indicator'] = 1
missing_df

In [ ]:
# Generate a complete time range with 1-minute frequency
# Instead of data['DateTime'], use data.index to access the DateTime values
full_range = pd.date_range(start=data.index.min(), end=data.index.max(), freq='T')

In [ ]:
# Create a dataframe from the complete time range
complete_df = pd.DataFrame(full_range, columns=['DateTime'])

# Merge the original dataframe with the complete time range
combined_df = pd.merge(complete_df, data, on='DateTime', how='left')

In [ ]:
# Merge the missing dataframe with the combined dataframe
combined_df = pd.merge(combined_df, missing_df, on='DateTime', how='left')

In [ ]:
combined_df

In [ ]:
# Fill the 'indicator' column: 0 for non-missing, 1 for missing
combined_df['indicator'] = combined_df['indicator'].fillna(0).astype(int)

# Sort by DateTime (optional)
combined_df = combined_df.sort_values(by='DateTime').reset_index(drop=True)

In [ ]:
combined_df.shape

In [ ]:
combined_df.iloc[35:45]

In [ ]:
combined_df.iloc[50:60]

In [ ]:
sample = combined_df.iloc[:75]

In [ ]:
# Group by date and sum the indicator column (missing minutes per day)
missing_per_day = sample.groupby(sample['DateTime'].dt.date)['indicator'].sum()

# Plot the bar chart
plt.figure(figsize=(15, 5))
missing_per_day.plot(kind='bar', color='red')
plt.title('Missing Data Per Day')
plt.xlabel('Date')
plt.ylabel('Number of Missing Minutes')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
# Group by date and sum the indicator column (missing minutes per day)
missing_per_day = combined_df.groupby(combined_df['DateTime'].dt.date)['indicator'].sum()

# Plot the bar chart
plt.figure(figsize=(35, 5))
missing_per_day.plot(kind='bar', color='red')
plt.title('Missing Data Per Day')
plt.xlabel('Date')
plt.ylabel('Number of Missing Minutes')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
# prompt: can i identify what times on the friday saturday and sunday i have missing data

# Assuming 'combined_df' is the DataFrame with the 'indicator' column (0 for present, 1 for missing) and 'DateTime' index.

# Filter for Friday, Saturday, and Sunday
combined_df['DayOfWeek'] = combined_df['DateTime'].dt.dayofweek  # Monday=0, Sunday=6
weekend_df = combined_df[(combined_df['DayOfWeek'] >= 4)] # 4,5,6 are Friday, Saturday, Sunday


# Group by date and sum the indicator to find missing data points on weekends
missing_weekend = weekend_df.groupby(weekend_df['DateTime'].dt.date)['indicator'].sum()

# Print the dates and number of missing data points for Friday, Saturday, and Sunday
for date, missing_count in missing_weekend.items():
    print(f"Date: {date}, Missing Count: {missing_count}")

# Alternatively, to get the specific times:
for date, missing_count in missing_weekend.items():
    if missing_count > 0:
        missing_times = weekend_df[(weekend_df['DateTime'].dt.date == date) & (weekend_df['indicator'] == 1)]
        print(f"\nMissing data on {date}:")
        print(missing_times['DateTime'])


In [ ]:
missing_per_day

In [ ]:
# Group by month and sum the indicator column (missing minutes per month)
missing_per_month = combined_df.groupby(combined_df['DateTime'].dt.month)['indicator'].sum()

# Plot the bar chart
plt.figure(figsize=(35, 5))
missing_per_month.plot(kind='bar', color='red')
plt.title('Missing Data Per Month')
plt.xlabel('Month')
plt.ylabel('Number of Missing Minutes')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
# We expect to have 4 weekends/ mo. Therefore 4*2*24*60 minutes missing per month = 11,520

In [ ]:
print(missing_per_month)

In [ ]:
sum(missing_per_month)-(11520*12)

In [ ]:
#1
# Rename columns for clarity
data.columns = ['Raw_DateTime', 'Open', 'High', 'Low', 'Close', 'Volume']

# Split the Raw_DateTime column into Date and Time
data[['Date', 'Time']] = data['Raw_DateTime'].str.split(' ', expand=True)
data = data.drop(columns=['Raw_DateTime'])

# Combine Date and Time into a proper Datetime column
data['Datetime'] = pd.to_datetime(data['Date'] + ' ' + data['Time'], format='%Y%m%d %H%M%S')
data = data.drop(columns=['Date', 'Time'])

# Sort by the Datetime column
data = data.sort_values(by='Datetime').reset_index(drop=True)

# Create a complete 1-minute time range
time_range = pd.date_range(start=data['Datetime'].min(), end=data['Datetime'].max(), freq='1T')

data.tail(20)


In [ ]:
# Find and display duplicate 'Datetime' values
duplicate_datetimes = data[data['Datetime'].duplicated(keep=False)]['Datetime']

print("Duplicate Datetime values:")
print(duplicate_datetimes)

# Optionally, you can also display the entire rows with duplicate 'Datetime' values:
print("\nRows with duplicate Datetime values:")
print(duplicate_datetimes)

In [ ]:
data.iloc[25927:25930]

In [ ]:
#####   review

#2
# Drop duplicates in 'Datetime' before setting it as the index
data = data.drop_duplicates(subset=['Datetime'])
# Reindex the dataset to match the complete time range
data = data.set_index('Datetime').reindex(time_range).reset_index()
data = data.rename(columns={'index': 'Datetime'})

# Handle missing values (choose one strategy)
# Forward fill missing values
data.fillna(method='ffill', inplace=True)

# Backward fill missing values
# data.fillna(method='bfill', inplace=True)

# Display the first few rows of the cleaned dataset
data.tail(20)

In [ ]:
import matplotlib.pyplot as plt

# Plot the 'Close' prices
plt.figure(figsize=(12, 6))
plt.plot(data['Datetime'], data['Close'], label='Close Price', linewidth=1)

# Enhancing the plot aesthetics
plt.title('EUR/USD Close Price Over Time', fontsize=16)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Close Price', fontsize=12)
plt.grid(alpha=0.3)
plt.legend(loc='upper left')
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
# Create a column to identify hourly groups
data['Hour'] = pd.to_datetime(data['Datetime']).dt.floor('h').dt.hour
data

In [ ]:
#######################################
########    CLOSE-CLOSE RVOL    #######

# Compute log returns
data['Log_Returns'] = np.log(data['Close'] / data['Close'].shift(1))

# Square the log returns
data['Squared_Returns'] = data['Log_Returns']**2

# Set 'Datetime' as the index before resampling
data = data.set_index('Datetime')  # Set 'Datetime' as index

# Resample the data to hourly groups and count the number of rows in each group
group_sizes = data['Squared_Returns'].resample('h').count() -1 # N-1
# Aggregate squared returns by hour (sum)
hourly_squared_sum = data['Squared_Returns'].resample('h').sum()
hourly_squared_sum
# Compute variance by dividing the sum by the group size
hourly_variance = hourly_squared_sum / group_sizes
hourly_variance
# Compute realized volatility as the square root of the variance
hourly_rvol = hourly_variance.apply(lambda x: x**0.5)
hourly_rvol.tail()

# Combine results into a DataFrame for easy analysis
hourly_rvol_df = pd.DataFrame({'Hourly_RVol': hourly_rvol})
hourly_rvol_df

In [ ]:
from matplotlib import pyplot as plt
hourly_rvol_df['Hourly_RVol'].iloc[-750:-100].plot(kind='line', figsize=(20, 4))
plt.gca().spines[['top', 'right']].set_visible(False)

plt.title("Hourly Realized Volatility (rVol)", fontsize=14, fontweight='bold')
plt.xlabel("Timestamp", fontsize=12)
plt.ylabel("Realized Volatility (rVol)", fontsize=12)
plt.grid(alpha=0.5)
plt.show()

In [ ]:
#hourly_rvol_df['Hourly_RVol'].iloc[-310:-100]

In [ ]:
V_ohlc= 0.5*(data["High"]-data["Low"])**2 -(2*np.log(2)-1)*(data["Close"]-data["Open"])**2

In [ ]:
# Different 60 min rvol/1 min price



# Constants
k = 2 * np.log(2) - 1  # Constant for Garman-Klass

# Garman-Klass Volatility Calculation
data['V_GK'] = 0.5 * (np.log(data['High'] / data['Low']) ** 2) - k * (np.log(data['Close'] / data['Open']) ** 2)

# Rogers-Satchell Volatility Calculation
data['V_RS'] = (np.log(data['High'] / data['Open']) * np.log(data['High'] / data['Close']) +
                np.log(data['Low'] / data['Open']) * np.log(data['Low'] / data['Close']))

# Close-to-Close Volatility Calculation
data['Log_Returns'] = np.log(data['Close'] / data['Close'].shift(1))
data['V_CC'] = data['Log_Returns'] ** 2  # Squared log returns for variance

# Resampling to hourly intervals
hourly_counts = data.resample('H').size()  # Number of data points per hour (N)
hourly_data = data.resample('H').agg({
    'V_GK': 'sum',
    'V_RS': 'sum',
    'V_CC': 'sum'
})

# Adjust by (N-1) for sample variance
hourly_data['V_GK'] /= (hourly_counts - 1)
hourly_data['V_RS'] /= (hourly_counts - 1)
hourly_data['V_CC'] /= (hourly_counts - 1)

# Compute realized volatility (square root of sample variances)
hourly_data['Hourly_RVol_GK'] = np.sqrt(hourly_data['V_GK'])
hourly_data['Hourly_RVol_RS'] = np.sqrt(hourly_data['V_RS'])
hourly_data['Hourly_RVol_CC'] = np.sqrt(hourly_data['V_CC'])

# Final DataFrame for analysis
hourly_rvol_df = hourly_data[['Hourly_RVol_GK', 'Hourly_RVol_RS', 'Hourly_RVol_CC']]
# Display the last few rows
print(hourly_rvol_df.tail())


In [ ]:

# Plot the data for the specified range
plt.figure(figsize=(20, 6))

# Plot Garman-Klass
plt.plot(hourly_rvol_df['Hourly_RVol_GK'].iloc[-750:-100], label='Garman-Klass', color='blue')

# Plot Rogers-Satchell
plt.plot(hourly_rvol_df['Hourly_RVol_RS'].iloc[-750:-100], label='Rogers-Satchell', color='orange')

# Plot Close-to-Close
plt.plot(hourly_rvol_df['Hourly_RVol_CC'].iloc[-750:-100], label='Close-to-Close', color='green')

# Customize the plot
plt.gca().spines[['top', 'right']].set_visible(False)
plt.title("Hourly Realized Volatility (rVol)", fontsize=14, fontweight='bold')
plt.xlabel("Timestamp", fontsize=12)
plt.ylabel("Realized Volatility (rVol)", fontsize=12)
plt.grid(alpha=0.5)

# Add legend
plt.legend(title="Volatility Measure", fontsize=10)

# Show the plot
plt.show()


In [ ]:
# Constants
k = 2 * np.log(2) - 1  # Constant for Garman-Klass

# Garman-Klass 1-Minute RVOL
data['RVOL_GK_1min'] = np.sqrt(0.5 * (np.log(data['High'] / data['Low']) ** 2) -
                          k * (np.log(data['Close'] / data['Open']) ** 2))

# Rogers-Satchell 1-Minute RVOL
data['RVOL_RS_1min'] = np.sqrt(np.log(data['High'] / data['Open']) * np.log(data['High'] / data['Close']) +
                          np.log(data['Low'] / data['Open']) * np.log(data['Low'] / data['Close']))

# Close-to-Close 1-Minute RVOL
data['Log_Returns'] = np.log(data['Close'] / data['Close'].shift(1))
data['RVOL_CC_1min'] = np.sqrt(data['Log_Returns'] ** 2)

# Display the last few rows
print(data[['Open', 'High', 'Low', 'Close','RVOL_GK_1min', 'RVOL_RS_1min', 'RVOL_CC_1min']].tail())


In [ ]:
data[['RVOL_GK_1min', 'RVOL_RS_1min', 'RVOL_CC_1min']].iloc[-124:-100].plot(figsize=(20, 6), title="1-Minute Realized Volatility")
plt.grid(alpha=0.5)
plt.xlabel("Timestamp")
plt.ylabel("RVOL")
plt.show()


In [ ]:
# Resample 1-minute data into 5-minute OHLC data
data_5min = data.resample('5T').agg({
    'Open': 'first',
    'High': 'max',
    'Low': 'min',
    'Close': 'last'
}).dropna()  # Drop intervals with missing data


In [ ]:

# Constants
k = 2 * np.log(2) - 1  # Constant for Garman-Klass

# Garman-Klass Variance Calculation
data_5min['V_GK'] = 0.5 * (np.log(data_5min['High'] / data_5min['Low']) ** 2) - \
                    k * (np.log(data_5min['Close'] / data_5min['Open']) ** 2)

# Rogers-Satchell Variance Calculation
data_5min['V_RS'] = (np.log(data_5min['High'] / data_5min['Open']) * np.log(data_5min['High'] / data_5min['Close']) +
                     np.log(data_5min['Low'] / data_5min['Open']) * np.log(data_5min['Low'] / data_5min['Close']))

# Close-to-Close Variance Calculation
data_5min['Log_Returns'] = np.log(data_5min['Close'] / data_5min['Close'].shift(1))
data_5min['V_CC'] = data_5min['Log_Returns'] ** 2

# Resample to 360-minute intervals (6 hours)
interval = '360T'  # 360 minutes
rvol_360min_data = data_5min.resample(interval).agg({
    'V_GK': 'sum',
    'V_RS': 'sum',
    'V_CC': 'sum',
    'Log_Returns': 'size'  # Count number of 5-min intervals (N)
})

# Adjust by (N-1) for sample variance
rvol_360min_data['V_GK'] /= (rvol_360min_data['Log_Returns'] - 1)
rvol_360min_data['V_RS'] /= (rvol_360min_data['Log_Returns'] - 1)
rvol_360min_data['V_CC'] /= (rvol_360min_data['Log_Returns'] - 1)

# Compute realized volatility (square root of adjusted variances)
rvol_360min_data['RVOL_360min_GK'] = np.sqrt(rvol_360min_data['V_GK'])
rvol_360min_data['RVOL_360min_RS'] = np.sqrt(rvol_360min_data['V_RS'])
rvol_360min_data['RVOL_360min_CC'] = np.sqrt(rvol_360min_data['V_CC'])

# Final DataFrame for analysis
rvol_360min_df = rvol_360min_data[['RVOL_360min_GK', 'RVOL_360min_RS', 'RVOL_360min_CC']]

# Display the last few rows
print(rvol_360min_df.tail())

In [ ]:
from datetime import datetime
# Reset the index to make 'Datetime' a regular column
data = data.reset_index()

# Now perform the join
data = data.join(hourly_rvol_df, on=pd.to_datetime(data['Datetime']).dt.floor('H'))

In [ ]:
from datetime import datetime

# Reset the index to make 'Datetime' a regular column
data = data.reset_index()

# Create a common join key column for hourly_rvol_df
data['JoinKey_Hourly'] = pd.to_datetime(data['Datetime']).dt.floor('H')
hourly_rvol_df['JoinKey_Hourly'] = hourly_rvol_df.index

# Create a common join key column for rvol_360min_df
data['JoinKey_360min'] = pd.to_datetime(data['Datetime']).dt.floor('360T')
rvol_360min_df['JoinKey_360min'] = rvol_360min_df.index

# Now perform the join using the common 'JoinKey' columns
data = data.merge(hourly_rvol_df, on='JoinKey_Hourly', how='left').merge(rvol_360min_df, on='JoinKey_360min', how='left')

# Optionally, drop the 'JoinKey' columns if not needed
data = data.drop(columns=['JoinKey_Hourly', 'JoinKey_360min'])

# Set 'Datetime' back as the index
# data = data.set_index('Datetime')  # Set 'Datetime' as index (Optional)

In [ ]:
data.info()

In [ ]:
data.info()

In [ ]:
# Reset the index to a DatetimeIndex if needed
data = data.set_index('Datetime')

In [ ]:
data[['Open','High', 'Low', 'Close','RVOL_360min_GK']].iloc[-300:-250]